In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import re

In [2]:
o3d.__version__

'0.16.1'

In [32]:
def get_subdirectories(folder_path):
    subdirectories = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subdirectories.append(item)
    return subdirectories

def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files


def extract_string(filename,format = 'stl'):
    pattern = r'\/([^/]+)\.'+format+'$'
    match = re.search(pattern, filename)
    if match:
        return match.group(1)
    else:
        return None
    
# Defining a function to convert degrees to radians.
def deg2rad(deg):
    return deg * np.pi/180

def get_rotated_pcd(pcd, x_theta, y_theta, z_theta):
    pcd_rotated = copy.deepcopy(pcd)
    R = pcd_rotated.get_rotation_matrix_from_axis_angle([x_theta, y_theta, z_theta])
    pcd_rotated.rotate(R, center=(0, 0, 0))
    return pcd_rotated

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
        return False
    else:
        print(f"Folder already exists: {folder_path}")
        return True

def delete_file(file_path):
    try:
        os.remove(file_path)
        # print(f"File '{file_path}' deleted successfully.")
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except PermissionError:
        print(f"Permission denied: unable to delete file '{file_path}'.")
    except Exception as e:
        print(f"An error occurred while deleting the file: {e}")


In [33]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
data_path = parent_dir+'/data/'
cad_path = data_path + 'cad-models/'
print(cad_path)
file_extension = '.stl'
path_save = data_path+'train/images3/'
print(path_save)
files = get_files_in_subdirectories(cad_path,file_extension)

exists = create_folder_if_not_exists(path_save)
if exists:
    images_del = get_files_in_subdirectories(path_save,'jpg')
    for imdel in images_del:
        delete_file(imdel)

for i_file in files:

    print("Testing IO for meshes ...")
    name_part = extract_string(i_file)
    print(name_part)
    
    mesh = o3d.io.read_triangle_mesh(i_file)
    mesh.compute_vertex_normals()
    print(mesh)

    # Create a material with a different color (you can customize this)
    material = o3d.visualization.Material()
    print(material)
    # material.base_color = (1.0, 0.0, 0.0)  # Red color

    # Defining the steps and range of angle values by which to rotate the point cloud.
    theta_range = np.linspace(-180, 180, 6)
    zoom_range  = np.linspace(1.2,7.5,4)

    # Counting the number of sequential operations.
    view_counter = 1
    total_views = theta_range.shape[0] ** 3

    i = 0
    j = 0
    k = 0
    l = 0
    # Looping through the angle values defined above for each axis.
    for x_theta_deg in theta_range:
        j = 0
        for y_theta_deg in theta_range:
            i = 0
            for z_theta_deg in theta_range:
                l=0
                for l_zoom in zoom_range:
                        
                    print(f"Taking image {view_counter} of {total_views}.")

                    # Rotating the point cloud by the given angle values.
                    x_theta = deg2rad(x_theta_deg)
                    y_theta = deg2rad(y_theta_deg)
                    z_theta = deg2rad(z_theta_deg)
                    pcd_rotated = get_rotated_pcd(mesh, x_theta, y_theta, z_theta)

                    # draw_geoms_list = [pcd_rotated]

                    vis = o3d.visualization.Visualizer()
                    vis.create_window(visible=False)

                    # Capture the depth and color images
                    # depth_image = vis.capture_depth_float_buffer()
                    # color_image = vis.capture_screen_float_buffer()

                    # print(depth_image)
                    # # Get the size of the images (width and height)
                    # depth_width, depth_height = depth_image.shape[1], depth_image.shape[0]
                    # color_width, color_height = color_image.shape[1], color_image.shape[0]

                    # print(f"Depth Image Size: {depth_width} x {depth_height}")
                    # print(f"Color Image Size: {color_width} x {color_height}")

                    # vis.add_geometry(mesh)
                    vis.add_geometry(pcd_rotated)
                    # pcd_rotated.translate((-20, 10, 0))  # Adjust the translation values as needed

                    # vis.get_view_control().translate(50, 100, 0)

                    # pcd_rotated.material = material  # Apply the custom material


                    # R = mesh.get_rotation_matrix_from_xyz((angle_vector[i],angle_vector[j],angle_vector[k]))
                    # mesh.rotate(R, center=(0, 0, 0))
                    vis.get_view_control().set_zoom(l_zoom)  # Adjust the zoom factor (0.5 zooms out, 2.0 zooms in)

                    vis.poll_events()
                    vis.update_renderer()
                    #vis.reset_view_point()
                    name_ijk = str(i)+'_'+str(j)+'_'+str(k)+'_'+str(l)
                    vis.capture_screen_image(filename = path_save+'img_ '+name_part+'_ '+name_ijk+'.jpg')

                    # o3d.visualization.draw_geometries(draw_geoms_list)
                    view_counter += 1
                    l +=1
                i +=1
            j +=1
        k+=1

    vis.destroy_window()

/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/cad-models/
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train/images2/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train/images2/
Testing IO for meshes ...
10_00_0125_0084a_v4a
TriangleMesh with 444 points and 296 triangles.
Taking image 1 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 2 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 3 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 4 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 5 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 6 of 216.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Taking image 7 of 216.
[Ope

In [34]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
data_path = parent_dir+'/data/'
cad_path = data_path + 'cad-models/'
print(cad_path)
file_extension = '.stl'
path_save = data_path+'train/images2/'
labels_save = data_path+'train/labels2/'
print(path_save)
files = get_files_in_subdirectories(cad_path,file_extension)

/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/cad-models/
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train/images2/


In [35]:
import cv2
images_move = get_files_in_subdirectories(path_save,'.jpg')

unique_classes = []

for jpg in images_move:
    # Load the image
    image = cv2.imread(jpg)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the convex hull of the contours
    convex_hull = cv2.convexHull(np.vstack(contours))

    # Create a copy of the image for drawing the bounding box
    result_image = image.copy()

    # Calculate the bounding box of the convex hull
    x, y, w, h = cv2.boundingRect(convex_hull)

    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h

    # Calculate the maximum x and y coordinates for the new location
    max_x = image.shape[1] - w
    max_y = image.shape[0] - h

    # Generate random x and y coordinates for the new location
    new_x = np.random.randint(0, max_x + 1)
    new_y = np.random.randint(0, max_y + 1)

    # Extract the object
    object_region = image[y1:y2, x1:x2].copy()

    # Paste the object into the new random location
    image[y1:y2, x1:x2] = [255, 255, 255]
    image[new_y:new_y+h, new_x:new_x+w] = object_region

    # Save the edited image
    cv2.imwrite(jpg, image)
    
    # Create a copy of the image for drawing the bounding box
    result_image = image.copy()

    # Draw the bounding box on the result image
    cv2.rectangle(result_image, (new_x, new_y), (new_x + w, new_y + h), (0, 255, 0), 2)
    bounding_box_saving_path = jpg.replace("images2", "images2_wbb")
    cv2.imwrite(bounding_box_saving_path, result_image)

    center_x = new_x + (w) / 2
    center_y = new_y + (h) / 2
    relative_width = (w) / image.shape[1]
    #print(relative_width)
    relative_height = (h) / image.shape[0]
    #print(relative_height)
    relative_center_x= center_x / image.shape[1]
    relative_center_y= center_y / image.shape[0]

    # Open a file for writing the results
    filename = jpg.split('/')[-1].split('.')[0]
    classname = filename.split('_ ')[1]
    if classname not in unique_classes:
        unique_classes.append(classname)
    encoded_class = unique_classes.index(classname)
    with open(labels_save + filename + '.txt', 'w') as file:
        file.write(f"{encoded_class} {relative_center_x:.5f} {relative_center_y:.5f} {relative_width:.5f} {relative_height:.5f}")
    